# 4.1. - Redes recurrentes

En este notebook, exploraremos uno de los tipos de Redes Neuronales Recurrentes (RNN) más utilizados: las Long Short-Term Memory (LSTM).
Estas están diseñadas para manejar **secuencias de datos** y resolver el problema del **desvanecimiento del gradiente** de las redes recurrentes básicas.

Como ya hemos visto en teoría, en una LSTM propagamos entre cada etapa dos vectores diferentes, el de estado y la salida en si. Para obtener dichos vectores, necesitamos calcular el valor de la puerta de actualización, la de olvido y la de salida.

Este proceso será transparente a nosotros dado que Pytorch ya nos proporciona una capa [`LSTM`](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html) y simplemente tendremos que preocuparnos de manejar la secuencia de entrada y la de salida. Además, gracias a los parámetros `num_layers` y `bidirectional` podremos crear redes mucho más complejas de forma sencilla.

## Conjunto de datos
Puesto que ahora trabajaremos con secuencias, para este ejemplo vamos a utilizar un conjunto de datos de texto. En este caso hemos optado por descargar el libro "Trafalgar" de Benito Pérez Galdós.

Como hemos explicado en teoría, existen muchos tipos de problemas que podemos resolver con secuencias, pero en este caso nos centraremos en una de las más sencillas de implementar: predecir la siguiente palabra en una secuencia de 5 palabras dadas.

### Descargar conjunto

In [ ]:
import requests

# Descargar el texto de "Trafalgar" de Benito Pérez Galdós
url = "https://www.gutenberg.org/cache/epub/16961/pg16961.txt"
response = requests.get(url, timeout=30)
text = response.text

### Preprocesar texto

Normalmente los conjuntos de datos contienen errores o elementos no deseados, en esta parte los eliminaremos. Algunos ejemplos típicos son los números, los saltos de línea `\n` o los tabuladores `\t`. <br>
Si abres la url del libro en tu navegador verás que al inicio y al final nos aparece un aviso de copyright que tendremos que eliminar.

En este punto verás que también separamos el texto en frases haciendo uso de la librería de texto `nltk`.

In [ ]:
import re
import time
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
from nltk.tokenize import sent_tokenize
from unidecode import unidecode

# Eliminar saltos de línea y retornos de carro
text = re.sub(r'[\n\r]+', ' ', text)
# Eliminamos cabecera y fin en inglés
text = text.split("FIN DE TRAFALGAR")[0]
text = text.split(" -I- ")[1]
# Dividir el texto en frases utilizando la librería nltk
sentences = sent_tokenize(text)
# Eliminar acentos utilizando la librería unidecode
sentences = [unidecode(s) for s in sentences]
# Pasar a minúsculas y eliminar resto de caracteres extraños (dos puntos, punto y coma, números...)
sentences = [re.sub(r'[^a-z\s]', '', s.lower()) for s in sentences]

### Obtener vocabulario

Una vez tenemos todas las frases, para codificar cada una de las palabras del texto es necesario obtener el llamado `corpus` o `vocabulario`. Este nos sirve para saber que palabras únicas (sin repetición) aparecen en nuestro texto, lo cual es útil para asignar un índice a cada una. Como verás, en este caso asignamos el índice en función de la frecuencia de aparición, por tanto, la palabra más frecuente tendrá el índice más bajo.

Como recordarás, nuestro objetivo es que, dado un texto de máximo 5 palabras, el modelo entrenado sea capaz de continuar la frase hasta que decida finalizarla. Para lograrlo tenemos que tener en cuenta varios escenarios:
* ¿Y si nos dan menos de 5 palabras?
* ¿Como hacemos para detectar que la palabra generada por el modelo es la última de la frase?
* ¿Si nos dan una palabra que no existe en nuestro vocabulario? Una en otro idioma, por ejemplo.

Para solventar estos problemas, vamos a añadir tres palabras nuevas o `tokens` a nuestro vocabulario:
* **\<pad\>**: Token que representa *padding*, es decir, si nos dan menos de 5 palabras, rellenaremos las que falten con este padding.
* **\<eos\>**: Token que representa *end-of-sequence*, es decir, fin de secuencia. Si el modelo predice este token, sabremos que ya se acabó la frase y podemos detener la generación.
* **\<unk\>**: Token que representa *unknown*, es decir, desconocido. Si una palabra no aparece en el vocabulario, el modelo la codificará con este token.


In [ ]:
from collections import Counter

# Crear una lista con todas las palabras del texto
words = " ".join(sentences).split()
# Obtener el corpus del texto (todas las palabras que aparecen y su frecuencia)
word_counts = Counter(words)
# Filtrar palabras que aparecen menos de 5 veces, dado que no aportan mucho
word_counts = {word: count for word, count in word_counts.items() if count >=5}
# Ordenamos el corpus por frecuencia
word_counts = dict(sorted(word_counts.items(), key=lambda x: x[1], reverse=True))
# Extraemos solo las claves del diccionario anterior para crear el corpus
vocab = list(word_counts.keys())
# Añadimos tokens especiales <pad> (padding), <eos> (end of sequence), y <unk> (unknown)
vocab = ["<pad>", "<unk>"] + vocab + ["<eos>"]
# Generamos un diccionario (y su inverso) donde asignamos un id a cada palabra según su frecuencia.
# La palabra más frecuente será la 1. El 0 lo dejamos para el padding
word_to_ix = {word: i for i, word in enumerate(vocab)}
ix_to_word = {i: word for i, word in enumerate(vocab)}
vocab_size = len(vocab)

print(f'Tamaño del vocabulario: {vocab_size}')
print(f'Word to Index mapping: {word_to_ix}')

### Análisis de datos
En todos los problemas de aprendizaje, es muy importante hacer un análisis de los datos con los que vamos a trabajar, lo cual nos puede servir para diagnosticar o evitar futuros problemas.<br>
En este caso realizamos un simple análisis de la frecuencia de las palabras. 

Como siempre suele suceder en estos casos, las palabras más comunes son las llamadas `stop-words`. Estas palabras son las que más solemos repetir en nuestros textos y las forman los artículos, las preposiciones, las conjunciones, ... 

In [ ]:
import matplotlib.pyplot as plt

# Visualización de las 20 palabras más comunes
words_x = list(word_counts.keys())[:20]
counts_y = list(word_counts.values())[:20]

plt.figure(figsize=(10, 6))
plt.bar(words_x, counts_y)
plt.title('Palabras más comunes')
plt.xlabel('Palabras')
plt.ylabel('Frecuencia')
plt.show()

### Crear dataset Pytorch
Una vez tenemos nuestro conjunto limpio y analizado, podemos crear los ejemplos con los que alimentaremos nuestro modelo. Como habíamos comentado queremos intentar completar una frase dadas, como máximo 5 palabras.<br>
En la siguiente clase `SentenceDataset` crearemos los ejemplos a partir de las frases del texto descargado. Estos ejemplos han de reflejar todos los escenarios a los que luego queremos que nuestro modelo pueda enfrentarse.

Por tanto, para cada frase, nuestro `Dataset` creará los siguientes ejemplos:
* Frase: *Frase de ejemplo*
    * [\<pad\>, \<pad\>, \<pad\>, \<pad\>, frase], de
    * [\<pad\>, \<pad\>, \<pad\>,  frase, de], ejemplo
    * [\<pad\>, \<pad\>, frase,  de, ejemplo], \<eos\>

El primer vector es la secuencia de entrada y el elemento del final es la salida deseada. Ten en cuenta que al modelo no le damos el texto, le daremos los índices de cada palabra.


Si nuestro `word_to_ix` fuese este:
```python
word_to_ix = {"<pad>":0, "frase":1, "de":2, "ejemplo":3, "<eos>":4}
```
Nuestros ejemplos quedarían así:
* [0, 0, 0, 0, 1], 2
* [0, 0, 0, 1, 2], 3
* [0, 0, 1, 2, 3], 4

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class SentenceDataset(Dataset):
    def __init__(self, sentences, word_to_ix, seq_length):
        self.data = []
        self.seq_length = seq_length
        self.word_to_ix = word_to_ix
        
        # El índice para <unk>
        unk_index = self.word_to_ix["<unk>"]
        
        # Para cada frase del conjunto de datos..
        for sentence in sentences:
            # Separamos la frase en una lista de palabras y al final se añadimos el token <eos>
            words = sentence.split() + ['<eos>']
            # Para cada palabra, creamos un ejemplo
            for i in range(len(words)-1):
                # Obtenemos las palabras de cada ejemplo
                seq_in = words[max(0, i+1 - seq_length):i+1]
                # Añadir padding si es necesario
                seq_in = ['<pad>'] * (seq_length - len(seq_in)) + seq_in
                # Obtenemos la salida deseada para la secuencia de entrada (la siguiente palabra)
                seq_out = words[i+1]
                # Convertir palabras a índices
                seq_in_indices = [self.word_to_ix.get(word, unk_index) for word in seq_in]
                seq_out_index = self.word_to_ix.get(seq_out, unk_index)
                # Almacenamos el ejemplo solo si seq_out no es <unk> para evitar que el modelo prediga <unk>
                if seq_out_index != unk_index:
                    self.data.append((torch.tensor(seq_in_indices, dtype=torch.long), torch.tensor(seq_out_index, dtype=torch.long)))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Obtenemos un ejemplo concreto (ya se han creado todos en el init)
        seq_in, seq_out = self.data[idx]
        return seq_in, seq_out
 
seq_length = 5
dataset = SentenceDataset(sentences, word_to_ix, seq_length)
print(len(dataset))

### Dividir el conjunto de datos
Dividimos el conjunto al completo en ``entrenamiento``, ``validación`` y ``test`` (80%,10%,10%) de forma aleatoria.

In [ ]:
# Fijar la semilla para obtener reproducibilidad
seed = 42
torch.manual_seed(seed)  # Semilla para PyTorch

# Dividir el dataset en entrenamiento, validación y prueba
train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

batch_size = 256

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## 4.1.3. - Modelo
Para este problema crearemos el modelo más sencillo posible:
* **Capa Embedding**: Para cada elemento de la entrada, aprenderá un embedding que lo representará en un espacio de dimensión `embed_size`.
* **Capa LSTM**: Procesa el embedding de cada elemento de la secuencia de entrada, lo proyecta en otro espacio de tamaño `hidden_size` y retorna la secuencia de salida.
    * Ojo, la capa LSTM retorna dos cosas, las salidas y los vectores de estado **en cada instante de tiempo**. En este caso ignoramos los estados y solo utilizamos la salida en el último instante.
    * Como ya hemos comentado en teoría, en estas redes se acumula toda la información de la secuencia en el último elemento. 
* **Capa Linear**: Proyecta el vector del último elemento de la salida de la LSTM en un espacio de tamaño `output_size`. En este caso `output_size` es igual al tamaño del vocabulario dado que estamos intentando resolver un problema de multi-clasificación.

Nótese que no aplicamos la función `softmax` dado que esta se aplica internamente en la `CrossEntropyLoss`.



In [ ]:
import torch
import torch.nn as nn

# Definir el modelo LSTM
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, output_size, seq_length):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)


    def forward(self, sentence):
        embeds = self.embedding(sentence)
        lstm_out, _ = self.lstm(embeds)
        # Usar solo la salida del último paso de tiempo
        last_hidden = lstm_out[:, -1, :]
        out = self.fc(last_hidden)
        return out

In [ ]:
# Hiperparámetros del modelo
embed_size = 64
hidden_size = 32
output_size = vocab_size
num_epochs = 50
learning_rate = 0.005

# Inicializar los modelos, loss function y optimizer
lstm_model = LSTMModel(vocab_size, embed_size, hidden_size, output_size, seq_length)
lstm_model.to(device)
criterion = nn.CrossEntropyLoss()
lstm_optimizer = optim.Adam(lstm_model.parameters(), lr=learning_rate)

## 4.1.4. - Entrenamiento

> **EJERCICIO:** A continuación, crea el bucle de entrenamiento para este modelo utilizando los hiper-parámetros definidos previamente. Recuerda obtener también la loss de validación.

> **EJERCICIO:** Obtén e imprime, al final de cada época, la accuracy del modelo en el conjunto de entrenamiento y validación. Utiliza la siguiente función: 

In [ ]:
# Función para calcular la precisión (accuracy) en un conjunto de datos
def calculate_accuracy(model, data_loader, K=1):
    model.eval()  # Configuramos el modelo en modo evaluación (no se actualizan los pesos)
    correct = 0  # Contador para predicciones correctas
    total = 0  # Contador para el número total de ejemplos

    with torch.no_grad():  # Desactivamos el cálculo de gradientes
        for inputs, targets in data_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)  # Hacemos una predicción con el modelo
            # Obtenemos las probabilidades de las clases (Top-K)
            topk = outputs.topk(K, dim=1)[1]
            # Creamos una máscara que indica si la etiqueta correcta está entre las top-K predicciones
            correct_mask = topk.eq(targets.view(-1, 1).expand_as(topk))
            correct += correct_mask.sum().item()  # Contamos las predicciones correctas
            total += targets.size(0)  # Contamos el número total de ejemplos

    accuracy = correct / total  # Calculamos la precisión como el porcentaje de predicciones correctas
    return accuracy